In [4]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import nltk
import ssl

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet') # wordnet is the most well known lemmatizer for english
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import cluster

[nltk_data] Downloading package punkt to /Users/kaveri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kaveri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kaveri/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/kaveri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
df_books_for_nlp = pd.read_csv('df_books_final.csv')

In [17]:
def tokenizer_and_remove_punctuation(row):
  tokens = word_tokenize(row['bag_of_words'])
  return [word.lower() for word in tokens if word.isalpha()]

df_books_for_nlp['tokenized'] = df_books_for_nlp.apply(tokenizer_and_remove_punctuation,axis=1)
df_books_for_nlp.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,tokenized
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,"[the, hunger, games, suzanne, collins, award, ..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,"[harry, potter, and, the, order, of, the, phoe..."
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,"[to, kill, a, mockingbird, harper, lee, prize,..."
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","[pride, and, prejudice, jane, austen, anna, qu..."
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,"[twilight, stephenie, meyer, peach, book, awar..."


In [18]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper() # gets first letter of POS categorization
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # get returns second argument if first key does not exist


In [20]:
# lemmatize with part of speech helpers

lemmatizer = WordNetLemmatizer()

def lemmatizer_with_pos(row):
  return [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in row['tokenized']]

df_books_for_nlp['lemmatized'] = df_books_for_nlp.apply(lemmatizer_with_pos,axis=1)
df_books_for_nlp.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,tokenized,lemmatized
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,"[the, hunger, games, suzanne, collins, award, ...","[the, hunger, game, suzanne, collins, award, n..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,"[harry, potter, and, the, order, of, the, phoe...","[harry, potter, and, the, order, of, the, phoe..."
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,"[to, kill, a, mockingbird, harper, lee, prize,...","[to, kill, a, mockingbird, harper, lee, prize,..."
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","[pride, and, prejudice, jane, austen, anna, qu...","[pride, and, prejudice, jane, austen, anna, qu..."
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,"[twilight, stephenie, meyer, peach, book, awar...","[twilight, stephenie, meyer, peach, book, awar..."


In [21]:
# remove stopwords

def remove_sw(row):
  return list(set(row['lemmatized']).difference(stopwords.words()))

df_books_for_nlp['no_stopwords'] = df_books_for_nlp.apply(remove_sw,axis=1)
df_books_for_nlp.head(10)

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,tokenized,lemmatized,no_stopwords
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,"[the, hunger, games, suzanne, collins, award, ...","[the, hunger, game, suzanne, collins, award, n...","[sentence, suzanne, live, mexico, humanity, cr..."
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,"[harry, potter, and, the, order, of, the, phoe...","[harry, potter, and, the, order, of, the, phoe...","[fate, poison, level, harry, boundless, govern..."
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,"[to, kill, a, mockingbird, harper, lee, prize,...","[to, kill, a, mockingbird, harper, lee, prize,...","[forty, cruelty, dramatic, american, hatred, p..."
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","[pride, and, prejudice, jane, austen, anna, qu...","[pride, and, prejudice, jane, austen, anna, qu...","[performance, introduction, anna, splendid, qu..."
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,"[twilight, stephenie, meyer, peach, book, awar...","[twilight, stephenie, meyer, peach, book, awar...","[australian, meyer, saga, category, thumb, hig..."
5,19063.The_Book_Thief,The Book Thief,NaN,Markus Zusak (Goodreads Author),4.37,Librarian's note: An alternate cover edition c...,English,9780375831003,"['Historical Fiction', 'Fiction', 'Young Adult...","['Liesel Meminger', 'Hans Hubermann', 'Rudy St...",...,96.0,"['Molching (Germany)', 'Germany']",https://i.gr-assets.com/images/S/compressed.ph...,1372809,14168,3.8,The Book Thief Markus Zusak (Goodreads Author)...,"[the, book, thief, markus, zusak, goodreads, a...","[the, book, thief, markus, zusak, goodreads, a...","[foster, superbly, write, pick, publish, busie..."
6,170448.Animal_Farm,Animal Farm,NaN,"George Orwell, Russell Baker (Preface), C.M. W...",3.95,Librarian's note: There is an Alternate Cover ...,English,9780451526342,"['Classics', 'Fiction', 'Dystopia', 

In [22]:
# put all this cleaning together

def re_blob(row):
  return " ".join(row['no_stopwords'])

df_books_for_nlp['clean_blob'] = df_books_for_nlp.apply(re_blob,axis=1)
df_books_for_nlp.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,setting,coverImg,bbeScore,bbeVotes,price,bag_of_words,tokenized,lemmatized,no_stopwords,clean_blob
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,"[the, hunger, games, suzanne, collins, award, ...","[the, hunger, game, suzanne, collins, award, n...","[sentence, suzanne, live, mexico, humanity, cr...",sentence suzanne live mexico humanity cruel ev...
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,"[harry, potter, and, the, order, of, the, phoe...","[harry, potter, and, the, order, of, the, phoe...","[fate, poison, level, harry, boundless, govern...",fate poison level harry boundless government d...
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,"[to, kill, a, mockingbird, harper, lee, prize,...","[to, kill, a, mockingbird, harper, lee, prize,...","[forty, cruelty, dramatic, american, hatred, p...",forty cruelty dramatic american hatred publish...
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","[pride, and, prejudice, jane, austen, anna, qu...","[pride, and, prejudice, jane, austen, anna, qu...","[performance, introduction, anna, splendid, qu...",performance introduction anna splendid quadril...
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,"[twilight, stephenie, meyer, peach, book, awar...","[twilight, stephenie, meyer, peach, book, awar...","[australian, meyer, saga, category, thumb, hig...",australian meyer saga category thumb high read...


In [23]:
#let's take only the most common 10000 words
bow_vect = CountVectorizer(max_features=10000)
# fit creates one entry for each different word seen
X = bow_vect.fit_transform(df_books_for_nlp['clean_blob']).toarray()

In [24]:
as_df = pd.DataFrame(X,columns=bow_vect.get_feature_names_out())
as_df.head()

,aar,aaron,abandon,abandonment,abbott,abby,abc,abduct,abduction,abdul,...,يقول,يكن,يكون,يمكن,يوسف,يوم,کتاب,که,یک,星雲賞
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
as_df.shape

(52424, 10000)

In [26]:
kmeans = cluster.KMeans(n_clusters=20,random_state=100)
kmeans.fit(X)
pred = kmeans.predict(X)

In [27]:
predict_df = pd.concat([df_books_for_nlp['title'],pd.DataFrame(pred,columns=['class'])],axis=1)
predict_df.head(50)

,title,class
0,The Hunger Games,10
1,Harry Potter and the Order of the Phoenix,10
2,To Kill a Mockingbird,10
3,Pride and Prejudice,16
4,Twilight,10
5,The Book Thief,10
6,Animal Farm,16
7,The Chronicles of Narnia,4
8,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,7
9,Gone with the Wind,16


In [28]:
predict_df_entire = pd.concat([df_books_for_nlp,pd.DataFrame(pred,columns=['class'])],axis=1)

In [29]:
# predict_df_entire.to_csv('predictions.csv', index=False)

In [ ]:
predict_df_entire.dtypes

In [21]:
import pandas as pd
import dotenv
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
load_dotenv()
predict_df_entire_2 = pd.read_csv('predictions.csv')

In [9]:
pw = os.getenv('SQL')
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [10]:
# predict_df_entire_2.to_sql(name='books', con=engine, schema='final', if_exists='replace', index=False)

52424

In [12]:
predict_df_entire_2.columns

Index(['bookId', 'title', 'series', 'author', 'rating', 'description',
       'language', 'isbn', 'genres', 'characters', 'bookFormat', 'edition',
       'pages', 'publisher', 'publishDate', 'firstPublishDate', 'awards',
       'numRatings', 'ratingsByStars', 'likedPercent', 'setting', 'coverImg',
       'bbeScore', 'bbeVotes', 'price', 'bag_of_words', 'tokenized',
       'lemmatized', 'no_stopwords', 'clean_blob', 'class'],
      dtype='object')

In [19]:
predict_df_entire_2.genres = predict_df_entire_2.genres.apply(str)

In [24]:
predict_df_entire_2.head().explode('genres')

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,coverImg,bbeScore,bbeVotes,price,bag_of_words,tokenized,lemmatized,no_stopwords,clean_blob,class
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09,The Hunger Games Suzanne Collins ['Locus Award...,"['the', 'hunger', 'games', 'suzanne', 'collins...","['the', 'hunger', 'game', 'suzanne', 'collins'...","['sentence', 'suzanne', 'live', 'mexico', 'hum...",sentence suzanne live mexico humanity cruel ev...,10
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38,Harry Potter and the Order of the Phoenix J.K....,"['harry', 'potter', 'and', 'the', 'order', 'of...","['harry', 'potter', 'and', 'the', 'order', 'of...","['fate', 'poison', 'level', 'harry', 'boundles...",fate poison level harry boundless government d...,10
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN,To Kill a Mockingbird Harper Lee ['Pulitzer Pr...,"['to', 'kill', 'a', 'mockingbird', 'harper', '...","['to', 'kill', 'a', 'mockingbird', 'harper', '...","['forty', 'cruelty', 'dramatic', 'american', '...",forty cruelty dramatic american hatred publish...,10
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN,"Pride and Prejudice Jane Austen, Anna Quindlen...","['pride', 'and', 'prejudice', 'jane', 'austen'...","['pride', 'and', 'prejudice', 'jane', 'austen'...","['performance', 'introduction', 'anna', 'splen...",performance introduction anna splendid quadril...,16
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1,Twilight Stephenie Meyer ['Georgia Peach Book ...,"['twilight', 'stephenie', 'meyer', 'peach', 'b...","['twilight', 'stephenie', 'meyer', 'peach', 'b...","['australian', 'meyer', 'saga', 'category', 't...",australian meyer saga category thumb high read...,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52419,11492014-fractured,Fractured,Fateful #2,Cheri Schmidt (Goodreads Author),4.00,The Fateful Trilogy continues with Fractured. ...,English,2940012616562,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",[],...,https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN,Fractured Cheri Schmidt (Goodreads Author) [] ...,"['fractured', 'cheri', 'schmidt', 'goodreads',...","['fracture', 'cheri', 'schmidt', 'goodreads', ...","['struggle', 'fateful', 'cure', 'ethan', 'surv...",struggle fateful cure ethan survival midst dis...,9
52420,11836711-anasazi,Anasazi,Sense of Truth #2,Emma Michaels,4.19,"'Anasazi', sequel to 'The Thirteenth Chime' by...",English,9999999999999,"['Mystery', 'Young Adult

In [28]:
"['Young Adult','Fantasy']".strip('[]').replace("'","").split(',')

['Young Adult', 'Fantasy']